In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
fields = ['nom_dept', 'nom_poll', 'unite', 'valeur']

In [3]:
# Auvergne-Rhone Alpes
path = 'aura' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0, usecols=fields)
    li.append(df)


aura_df = pd.concat(li, axis=0, ignore_index=True, sort=True)
aura_df.sample(5)

,nom_dept,nom_poll,unite,valeur
266,Rhône,Particules PM10,µg/m3,20.6
120,Savoie,Monoxyde d'azote,µg/m3,3.0
67,Puy-de-Dôme,Monoxyde d'azote,µg/m3,31.0
132,Haute-Savoie,Monoxyde d'azote,µg/m3,NaN
3,Loire,Oxydes d'azote,µg/m3,8.0


In [4]:
# Paris Ile de France
path = 'idf' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0, usecols=fields)
    li.append(df)

idf_df = pd.concat(li, axis=0, ignore_index=True, sort=True)
idf_df.sample(5)

,nom_dept,nom_poll,unite,valeur
50,YVELINES,O3,ug.m-3,48.0
208,PARIS,PM2.5,ug.m-3,18.2
214,YVELINES,PM2.5,ug.m-3,11.8
143,SEINE-SAINT-DENIS,PM10,ug.m-3,42.4
79,SEINE-ET-MARNE,O3,ug.m-3,46.0


In [5]:
# Grand est
est_df = pd.read_csv('mes_atmo_grand_est_annuel_poll_princ.csv', index_col=None, header=0, usecols=fields)
est_df.sample(5)

,nom_dept,nom_poll,valeur,unite
1802,Haut-Rhin,Dioxyde d'azote,28.0,µg/m3
2384,Moselle,Particules PM10,NaN,µg/m3
2555,Bas-Rhin,NaN,NaN,µg/m3
2217,Marne,Oxydes d'azote,NaN,µg/m3
2132,Bas-Rhin,NaN,NaN,µg/m3


In [6]:
# Haut de France
hdf_df = pd.read_csv('mes_hdf_annuel_poll_princ.csv', index_col=None, header=0, usecols=fields)
hdf_df.sample(5)

,nom_dept,nom_poll,valeur,unite
193,OISE,Particules PM10,18.80,ug.m-3
234,NORD,Benzène,0.63,ug.m-3
789,NORD,Particules fines PM2.5,NaN,ug.m-3
707,NORD,Ozone,NaN,ug.m-3
464,NORD,Particules PM10,15.80,ug.m-3


In [7]:
# PACA
paca_df = pd.read_csv('mes_sudpaca_annuelle.csv', index_col=None, header=0, usecols=fields)
paca_df.sample(5)

,nom_dept,nom_poll,valeur,unite
729,HAUTES-ALPES,"Particules fines PM2,5",11.1,ug.m-3
545,ALPES-MARITIMES,Dioxyde d'azote,27.5,ug.m-3
651,HAUTES-ALPES,Ozone,45.0,ug.m-3
84,BOUCHES-DU-RHONE,Dioxyde de soufre,2.0,ug.m-3
496,VAUCLUSE,Ozone,64.5,ug.m-3


In [8]:
# Super merging
merged_df = pd.concat([aura_df, idf_df, est_df, hdf_df, paca_df], axis=0, ignore_index=True, sort=True)

# We need to clean the polluant name here:
pol_cleaner = {"Dioxyde d'azote": "NO2", "Monoxyde d'azote": "NO", "Oxydes d'azote": "NO2", "Ozone": "O3", \
              "Particules PM10": "PM10", "Particules PM2,5": "PM2.5", "Particules fines PM2,5": "PM2.5", "Particules fines PM2.5": "PM2.5"}

merged_df = merged_df.replace({"nom_poll": pol_cleaner})

# Lowercase and sort by department name
merged_df['nom_dept'] = merged_df['nom_dept'].str.lower()

# Aggregate the values to have a global average over the years:
agg_df = merged_df.groupby(['nom_dept', 'nom_poll'],as_index=False).agg({'valeur': 'mean'})


# Pivot to extract the features of pollution
FR_pol_df = agg_df.pivot(index='nom_dept', columns='nom_poll', values='valeur')
FR_pol_df = FR_pol_df[['NO2', 'O3', 'PM10', 'PM2.5']]

# Missing data, source: https://www.atmo-auvergnerhonealpes.fr/sites/ra/files/atoms/files/bilanqa2017_09-zoom01-ain_vfinale_0.pdf
# https://www.atmo-auvergnerhonealpes.fr/sites/ra/files/atoms/files/dossier_presse_2018_v5v2.pdf
FR_pol_df.iloc[0,3] = 10 # Ain, PM2.5
FR_pol_df.iloc[6,3] = 10 # Ardeche, PM2.5
FR_pol_df.iloc[6,0] = 10 # Ardeche, NO2
FR_pol_df.iloc[10,3] = 7.5 # Cantal, PM2.5
FR_pol_df.iloc[14,3] = 7.5 # haute-loire, PM2.5

# http://www.aube.gouv.fr/content/download/19915/131913/file/18NIF023%20Aube%20Bedding%20-%20DLE%20-%20v2.pdf
FR_pol_df.iloc[7,3] = 14 # haute-loire, PM2.5

FR_pol_df

nom_poll,NO2,O3,PM10,PM2.5
nom_dept,,,,
ain,12.350000,52.700000,16.250000,10.000000
aisne,20.714286,48.750000,17.600000,14.000000
allier,11.587500,56.233333,12.166667,7.800000
alpes-de-haute-provence,13.675000,73.270000,11.080000,7.420000
alpes-maritimes,40.071429,62.155882,24.973913,13.383333
ardennes,12.945946,50.571429,18.789474,9.500000
ardèche,10.000000,31.000000,20.000000,10.000000
aube,19.266667,48.550000,19.000000,14.000000
bas-rhin,36.015873,50.640000,22.285714,14.636364


In [9]:
# By Hand deptartment number and population size:
pop_df = pd.DataFrame()
pop_df['nom_dept'] = FR_pol_df.index
pop_df['# dept'] = pd.Series([1, 2, 3, 4, 6, 8, 7, 10, 67, 13, 15, 26, 91, 68, 43, 52, 74, 5, 92,
                             38, 42, 51, 54, 55, 57, 59, 60, 75, 62, 63, 69, 73, 77, 93, 80, 95, 94,
                             83, 84, 88, 78])
pop_df['population'] = pd.Series([638425, 536136, 339384, 161588, 1083310, 273579, 325712, 310020, 
                                  1125559, 2024162, 145143, 511553, 1296130, 764030, 227283, 175640, 
                                  807360,  141284, 1609306, 1258722, 762941, 568895, 733481, 187187,
                                 1043522, 2604361, 824503, 12568755, 1468018, 653742, 457392, 431174,
                                 1403997, 1623111, 572443, 1228618, 1387926, 1058740, 559479, 367673, 1438266 ])

# Merging
FR_final_df = pd.merge(FR_pol_df, right=pop_df, left_index=True, right_on='nom_dept')
FR_final_df.dropna().to_csv("FR_pol.csv", index=False)
FR_final_df.dropna()

,NO2,O3,PM10,PM2.5,nom_dept,# dept,population
0,12.350000,52.700000,16.250000,10.000000,ain,1,638425
1,20.714286,48.750000,17.600000,14.000000,aisne,2,536136
2,11.587500,56.233333,12.166667,7.800000,allier,3,339384
3,13.675000,73.270000,11.080000,7.420000,alpes-de-haute-provence,4,161588
4,40.071429,62.155882,24.973913,13.383333,alpes-maritimes,6,1083310
5,12.945946,50.571429,18.789474,9.500000,ardennes,8,273579
6,10.000000,31.000000,20.000000,10.000000,ardèche,7,325712
7,19.266667,48.550000,19.000000,14.000000,aube,10,310020
8,36.015873,50.640000,22.285714,14.636364,bas-rhin,67,1125559
9,36.666364,61.508696,22.393617,12.245455,bouches-du-rhone,13,2024162
